In [27]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer,BertTokenizer, BertForMaskedLM
import torch
import pandas as pd
import numpy as np
import math
from tqdm import tqdm

In [6]:
raw_df = pd.read_json("../../datasets/eli5_clean_combined.jsonl", lines=True)
raw_df

,question,human_answers,chatgpt_answers,index,scraped_answers
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...,NaN,NaN
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...,NaN,NaN
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...,NaN,NaN
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...,NaN,NaN
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...,NaN,NaN
...,...,...,...,...,...
34351,What is closed-loop verification?,[Let's pretend that it's purge night and you a...,NaN,NaN,[Closed-loop verification is a process used in...
34352,Why is mustard oil illegal to consume in the US?,[Erucic acid is poison so there is a blanket b...,NaN,NaN,[Mustard oil is not illegal to consume in the ...
34353,How space probes shoot pictures back at Earth,[The technology to transmit information has be...,NaN,NaN,[Space probes shoot pictures back at Earth usi...
34354,What exactly do international waters mean for ...,[Every ship has a country of registry (flag st...,NaN,NaN,"[International waters, also known as the high ..."


In [7]:
# shift the data in scraped_answers to chatgpt_answers
for index, row in raw_df.iterrows():
    # check if chatgpt_answers is nan
    if pd.isnull(row["chatgpt_answers"]).all() if type(row["chatgpt_answers"]) == list else pd.isnull(row["chatgpt_answers"]):
      raw_df.at[index,"chatgpt_answers"] = row['scraped_answers']

raw_df

,question,human_answers,chatgpt_answers,index,scraped_answers
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...,NaN,NaN
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...,NaN,NaN
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...,NaN,NaN
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...,NaN,NaN
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...,NaN,NaN
...,...,...,...,...,...
34351,What is closed-loop verification?,[Let's pretend that it's purge night and you a...,[Closed-loop verification is a process used in...,NaN,[Closed-loop verification is a process used in...
34352,Why is mustard oil illegal to consume in the US?,[Erucic acid is poison so there is a blanket b...,[Mustard oil is not illegal to consume in the ...,NaN,[Mustard oil is not illegal to consume in the ...
34353,How space probes shoot pictures back at Earth,[The technology to transmit information has be...,[Space probes shoot pictures back at Earth usi...,NaN,[Space probes shoot pictures back at Earth usi...
34354,What exactly do international waters mean for ...,[Every ship has a country of registry (flag st...,"[International waters, also known as the high ...",NaN,"[International waters, also known as the high ..."


In [12]:
# drop index col which is all NaN
processed_hc3_df = raw_df.drop(["scraped_answers"], axis=1)
processed_hc3_df = processed_hc3_df.drop(["index"], axis=1)
processed_hc3_df.dropna(inplace=True)
processed_hc3_df

,question,human_answers,chatgpt_answers
0,"Why is every book I hear about a "" NY Times # ...","[Basically there are many categories of "" Best...",[There are many different best seller lists th...
1,"If salt is so bad for cars , why do we use it ...",[salt is good for not dying in car crashes and...,[Salt is used on roads to help melt ice and sn...
2,Why do we still have SD TV channels when HD lo...,[The way it works is that old TV stations got ...,[There are a few reasons why we still have SD ...
3,Why has nobody assassinated Kim Jong - un He i...,[You ca n't just go around assassinating the l...,[It is generally not acceptable or ethical to ...
4,How was airplane technology able to advance so...,[Wanting to kill the shit out of Germans drive...,[After the Wright Brothers made the first powe...
...,...,...,...
34351,What is closed-loop verification?,[Let's pretend that it's purge night and you a...,[Closed-loop verification is a process used in...
34352,Why is mustard oil illegal to consume in the US?,[Erucic acid is poison so there is a blanket b...,[Mustard oil is not illegal to consume in the ...
34353,How space probes shoot pictures back at Earth,[The technology to transmit information has be...,[Space probes shoot pictures back at Earth usi...
34354,What exactly do international waters mean for ...,[Every ship has a country of registry (flag st...,"[International waters, also known as the high ..."


In [14]:
# keep rows with non empty chatgpt ans
has_chatgpt_ans = processed_hc3_df["chatgpt_answers"].map(lambda ans_lst:ans_lst!=[])
processed_hc3_df = processed_hc3_df[has_chatgpt_ans]

#Need to expand the answers and chatgpt ans -- as for one question, there can be several human answers obtained
row_list = []
for index, row in processed_hc3_df.iterrows():
  human_ans_list = row["human_answers"]
  chatgpt_ans_list = row["chatgpt_answers"]
  # note that sometimes there can be 2 human ans, and 1 chatgpt ans for e.g. 
  num_chatgpt_ans = len(chatgpt_ans_list)
  for i in range(len(human_ans_list)):
    human_ans = human_ans_list[i]
    if i >= num_chatgpt_ans:
      chatgpt_ans = chatgpt_ans_list[num_chatgpt_ans-1]
    else:
      chatgpt_ans = chatgpt_ans_list[i]
    row_list.append([row["question"],human_ans,chatgpt_ans])

processed_hc3_df = pd.DataFrame(row_list, columns = ["question", "human_answer", "chatgpt_answer"])
processed_hc3_df.head()

,question,human_answer,chatgpt_answer
0,"Why is every book I hear about a "" NY Times # ...","Basically there are many categories of "" Best ...",There are many different best seller lists tha...
1,"Why is every book I hear about a "" NY Times # ...","If you 're hearing about it , it 's because it...",There are many different best seller lists tha...
2,"Why is every book I hear about a "" NY Times # ...","One reason is lots of catagories . However , h...",There are many different best seller lists tha...
3,"If salt is so bad for cars , why do we use it ...",salt is good for not dying in car crashes and ...,Salt is used on roads to help melt ice and sno...
4,"If salt is so bad for cars , why do we use it ...","In Minnesota and North Dakota , they tend to u...",Salt is used on roads to help melt ice and sno...


In [18]:
# combine the human and chatgpt ans
chatgpt_responses = processed_hc3_df.iloc[:, 0:3:2]
chatgpt_responses["label"]=1
chatgpt_responses.rename(columns = {'chatgpt_answer':'response'}, inplace = True)
chatgpt_responses.drop_duplicates(inplace=True)
print(chatgpt_responses.shape)
chatgpt_responses.head()

(33400, 3)


,question,response,label
0,"Why is every book I hear about a "" NY Times # ...",There are many different best seller lists tha...,1
3,"If salt is so bad for cars , why do we use it ...",Salt is used on roads to help melt ice and sno...,1
6,Why do we still have SD TV channels when HD lo...,There are a few reasons why we still have SD (...,1
9,Why has nobody assassinated Kim Jong - un He i...,It is generally not acceptable or ethical to a...,1
12,How was airplane technology able to advance so...,After the Wright Brothers made the first power...,1


In [20]:
human_responses = processed_hc3_df.iloc[:, 0:2]
human_responses["label"]=0
human_responses.rename(columns = {'human_answer':'response'}, inplace = True)
human_responses.drop_duplicates(inplace=True)
print(human_responses.shape)
human_responses.head()

(67123, 3)


,question,response,label
0,"Why is every book I hear about a "" NY Times # ...","Basically there are many categories of "" Best ...",0
1,"Why is every book I hear about a "" NY Times # ...","If you 're hearing about it , it 's because it...",0
2,"Why is every book I hear about a "" NY Times # ...","One reason is lots of catagories . However , h...",0
3,"If salt is so bad for cars , why do we use it ...",salt is good for not dying in car crashes and ...,0
4,"If salt is so bad for cars , why do we use it ...","In Minnesota and North Dakota , they tend to u...",0


In [22]:
Corpus = pd.concat([chatgpt_responses, human_responses], ignore_index=True)
Corpus

,question,response,label
0,"Why is every book I hear about a "" NY Times # ...",There are many different best seller lists tha...,1
1,"If salt is so bad for cars , why do we use it ...",Salt is used on roads to help melt ice and sno...,1
2,Why do we still have SD TV channels when HD lo...,There are a few reasons why we still have SD (...,1
3,Why has nobody assassinated Kim Jong - un He i...,It is generally not acceptable or ethical to a...,1
4,How was airplane technology able to advance so...,After the Wright Brothers made the first power...,1
...,...,...,...
100518,What is closed-loop verification?,Let's pretend that it's purge night and you ar...,0
100519,Why is mustard oil illegal to consume in the US?,Erucic acid is poison so there is a blanket ba...,0
100520,How space probes shoot pictures back at Earth,The technology to transmit information has bee...,0
100521,What exactly do international waters mean for ...,Every ship has a country of registry (flag sta...,0


In [26]:
class AbstractLanguageChecker():
    """
    Abstract Class that defines the Backend API of GLTR.

    To extend the GLTR interface, you need to inherit this and
    fill in the defined functions.
    """

    def __init__(self):
        '''
        In the subclass, you need to load all necessary components
        for the other functions.
        Typically, this will comprise a tokenizer and a model.
        '''
        self.device = torch.device(
            "cuda" if torch.cuda.is_available() else "cpu")

    def check_probabilities(self, in_text, topk=40):
        '''
        Function that GLTR interacts with to check the probabilities of words

        Params:
        - in_text: str -- The text that you want to check
        - topk: int -- Your desired truncation of the head of the distribution

        Output:
        - payload: dict -- The wrapper for results in this function, described below

        Payload values
        ==============
        bpe_strings: list of str -- Each individual token in the text
        real_topk: list of tuples -- (ranking, prob) of each token
        pred_topk: list of list of tuple -- (word, prob) for all topk
        '''
        raise NotImplementedError

    def postprocess(self, token):
        """
        clean up the tokens from any special chars and encode
        leading space by UTF-8 code '\u0120', linebreak with UTF-8 code 266 '\u010A'
        :param token:  str -- raw token text
        :return: str -- cleaned and re-encoded token text
        """
        raise NotImplementedError


def top_k_logits(logits, k):
    '''
    Filters logits to only the top k choices
    from https://github.com/huggingface/pytorch-pretrained-BERT/blob/master/examples/run_gpt2.py
    '''
    if k == 0:
        return logits
    values, _ = torch.topk(logits, k)
    min_values = values[:, -1]
    return torch.where(logits < min_values,
                       torch.ones_like(logits, dtype=logits.dtype) * -1e10,
                       logits)


class LM(AbstractLanguageChecker):
    def __init__(self, model_name_or_path="gpt2"):
        super(LM, self).__init__()
        self.enc = GPT2Tokenizer.from_pretrained(
            model_name_or_path, truncation=True, max_length=1024)
        self.model = GPT2LMHeadModel.from_pretrained(model_name_or_path)
        self.model.to(self.device)
        self.model.eval()
        self.start_token = '<|endoftext|>'
        print("Loaded GPT-2 model!")

    def check_probabilities(self, in_text, topk=40):
        try:
            # Process input
            start_t = torch.full((1, 1),
                                 self.enc.encoder[self.start_token],
                                 device=self.device,
                                 dtype=torch.long)
            context = self.enc.encode(in_text)
            # Limit is 1024 but we are adding a start token
            if len(context) > 1023:
                context = context[:1023]
            assert len(context) <= 1023
            context = torch.tensor(context, device=self.device,
                                   dtype=torch.long).unsqueeze(0)
            context = torch.cat([start_t, context], dim=1)
            # Forward through the model
            outputs = self.model(context)
            logits = outputs.logits
            # construct target and pred
            yhat = torch.softmax(logits[0, :-1], dim=-1)
            y = context[0, 1:]
            # Sort the predictions for each timestep
            sorted_preds = np.argsort(-yhat.data.cpu().numpy())
            # [(pos, prob), ...]
            real_topk_pos = list(
                [int(np.where(sorted_preds[i] == y[i].item())[0][0])
                 for i in range(y.shape[0])])
            real_topk_probs = yhat[np.arange(
                0, y.shape[0], 1), y].data.cpu().numpy().tolist()
            real_topk_probs = list(map(lambda x: round(x, 5), real_topk_probs))
            real_topk = list(zip(real_topk_pos, real_topk_probs))
            # [str, str, ...]
            bpe_strings = [self.enc.decoder[s.item()] for s in context[0]]

            bpe_strings = [self.postprocess(s) for s in bpe_strings]

            # [[(pos, prob), ...], [(pos, prob), ..], ...]
            pred_topk = [
                list(zip([self.enc.decoder[p] for p in sorted_preds[i][:topk]],
                         list(map(lambda x: round(x, 5),
                                  yhat[i][sorted_preds[i][
                                          :topk]].data.cpu().numpy().tolist()))))
                for i in range(y.shape[0])]

            pred_topk = [[(self.postprocess(t[0]), t[1])
                          for t in pred] for pred in pred_topk]
            payload = {'bpe_strings': bpe_strings,
                       'real_topk': real_topk,
                       'pred_topk': pred_topk}
            if torch.cuda.is_available():
                torch.cuda.empty_cache()

            return payload
        except IndexError as e:
            print(context)
            print(context[0])
            print(f"Context length: {len(context)}")
            print(f"Context token length: {len(context[0])}")
            print(in_text)

    def sample_unconditional(self, length=100, topk=5, temperature=1.0):
        '''
        Sample `length` words from the model.
        Code strongly inspired by
        https://github.com/huggingface/pytorch-pretrained-BERT/blob/master/examples/run_gpt2.py

        '''
        context = torch.full((1, 1),
                             self.enc.encoder[self.start_token],
                             device=self.device,
                             dtype=torch.long)
        prev = context
        output = context
        past = None
        # Forward through the model
        with torch.no_grad():
            for i in range(length):
                logits, past = self.model(prev, past=past)
                logits = logits[:, -1, :] / temperature
                # Filter predictions to topk and softmax
                probs = torch.softmax(top_k_logits(logits, k=topk),
                                      dim=-1)
                # Sample
                prev = torch.multinomial(probs, num_samples=1)
                # Construct output
                output = torch.cat((output, prev), dim=1)

        output_text = self.enc.decode(output[0].tolist())
        return output_text

    def postprocess(self, token):
        with_space = False
        with_break = False
        if token.startswith('Ġ'):
            with_space = True
            token = token[1:]
            # print(token)
        elif token.startswith('â'):
            token = ' '
        elif token.startswith('Ċ'):
            token = ' '
            with_break = True

        token = '-' if token.startswith('â') else token
        token = '“' if token.startswith('ľ') else token
        token = '”' if token.startswith('Ŀ') else token
        token = "'" if token.startswith('Ļ') else token

        if with_space:
            token = '\u0120' + token
        if with_break:
            token = '\u010A' + token

        return token


"""
From GLTR paper: By default, a word that ranks
within the top 10 is highlighted in green, top 100
in yellow, top 1,000 in red, and the rest in purple. 
"""


def bucketize_ranks(ranks):
    # 1-10 is category 1,top 101-1000 is category 3
    bins = np.array([0, 10, 100, 1000, math.inf])
    total_words = len(ranks)
    raw = np.histogram(ranks, bins)[0]
    processed = raw / total_words
    selected_categories = [processed[0], processed[2]]
    return selected_categories


def get_gltr_feature(text, language_model):
    payload = language_model.check_probabilities(text, topk=5)
    real_topK = payload["real_topk"]
    ranks = [i[0] for i in real_topK]
    return bucketize_ranks(ranks)


In [32]:
n = len(Corpus)
lm = LM()

for index, row in tqdm(Corpus.iterrows(),
               desc="Loading…",
               ascii=False):
  text = row["response"]
  gltr_arr = get_gltr_feature(text, lm)
  Corpus.at[index, "GLTR Category 1"] = gltr_arr[0]
  Corpus.at[index, "GLTR Category 3"] = gltr_arr[1]
  
Corpus

Loaded GPT-2 model!


Loading…: 33424it [8:24:10,  2.33it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (2026 > 1024). Running this sequence through the model will result in indexing errors
Loading…: 100523it [19:19:29,  1.44it/s]


,question,response,label,GLTR Category 1,GLTR Category 3
0,"Why is every book I hear about a "" NY Times # ...",There are many different best seller lists tha...,1,0.888412,0.017167
1,"If salt is so bad for cars , why do we use it ...",Salt is used on roads to help melt ice and sno...,1,0.816594,0.026201
2,Why do we still have SD TV channels when HD lo...,There are a few reasons why we still have SD (...,1,0.865169,0.011236
3,Why has nobody assassinated Kim Jong - un He i...,It is generally not acceptable or ethical to a...,1,0.793939,0.036364
4,How was airplane technology able to advance so...,After the Wright Brothers made the first power...,1,0.843478,0.017391
...,...,...,...,...,...
100518,What is closed-loop verification?,Let's pretend that it's purge night and you ar...,0,0.639706,0.110294
100519,Why is mustard oil illegal to consume in the US?,Erucic acid is poison so there is a blanket ba...,0,0.590278,0.131944
100520,How space probes shoot pictures back at Earth,The technology to transmit information has bee...,0,0.698413,0.079365
100521,What exactly do international waters mean for ...,Every ship has a country of registry (flag sta...,0,0.677419,0.193548


In [33]:
Corpus.to_csv("scraped_gltr_features.csv")

In [34]:
Corpus.iloc[33422:33425, :]

,question,response,label,GLTR Category 1,GLTR Category 3
33422,What has changed that we frequently now throw ...,planned obsolescence is the term . You can eng...,0,0.621951,0.109756
33423,What has changed that we frequently now throw ...,Plastics . If you take a look at a lot of olde...,0,0.700000,0.075000
33424,magic the gathering What is it . how popular i...,"EDIT , Nov 21 : By popular demand , now includ...",0,0.739003,0.069404
